In [71]:
%matplotlib inline
import os
import sys
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import torchvision
from torchvision import models

import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset
import torch.optim as optim
from torchmetrics import Accuracy
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
from torchvision.datasets import VOCDetection

sys.path.append('../')  
from Object_Detection import selective_search
from Object_Detection.bbox import calculate_iou
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np

# Settings 

torch.set_printoptions(precision=3)

In [72]:
voc_dataset = VOCDetection(root='./data', 
                           year='2012', 
                           image_set='train',
                           download=True, 
                        #    transform=transform
                           )

Using downloaded and verified file: ./data\VOCtrainval_11-May-2012.tar
Extracting ./data\VOCtrainval_11-May-2012.tar to ./data


# Load model and its finetuned weights (PASCAL VOC)

In [ ]:
# Step 1: Load the ResNet50 model without pretrained weights
resnet50 = models.resnet50(weights=False)

resnet50.fc = torch.nn.Linear(resnet50.fc.in_features, 20)

In [74]:
# Step 2: Load your finetuned weights
# Make sure that the path 'voc_finetune_resnet_weights.pth' is correct and accessible
resnet50.load_state_dict(torch.load('resnet50_pascal_voc.pth'))

<All keys matched successfully>

In [ ]:
summary(model=resnet50, input_size=(1, 3, 227, 227), col_width=20,
                  col_names=['input_size', 'output_size', 'num_params', 'trainable'], row_settings=['var_names'], verbose=0)

In [75]:
class ResNet50Modified(nn.Module):
    def __init__(self, original_model):
        super(ResNet50Modified, self).__init__()
        # Adapt this line if your model was changed differently
        self.features = nn.Sequential(*list(original_model.children())[:-1])  # Typically removes the final fully connected layer and avg pooling layer

    def forward(self, x):
        x = self.features(x)
        return x

# Create the modified model with loaded weights
modified_resnet50 = ResNet50Modified(resnet50)

# Put the model in evaluation mode if you are doing inference or feature extraction
modified_resnet50.eval()

if torch.cuda.is_available():
    modified_resnet50 = modified_resnet50.cuda()

In [57]:
summary(model=modified_resnet50, input_size=(1, 3, 227, 227), col_width=20,
                  col_names=['input_size', 'output_size', 'num_params', 'trainable'], row_settings=['var_names'], verbose=0)

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
ResNet50Modified (ResNet50Modified)           [1, 3, 227, 227]     [1, 2048, 1, 1]      --                   True
├─Sequential (features)                       [1, 3, 227, 227]     [1, 2048, 1, 1]      --                   True
│    └─Conv2d (0)                             [1, 3, 227, 227]     [1, 64, 114, 114]    9,408                True
│    └─BatchNorm2d (1)                        [1, 64, 114, 114]    [1, 64, 114, 114]    128                  True
│    └─ReLU (2)                               [1, 64, 114, 114]    [1, 64, 114, 114]    --                   --
│    └─MaxPool2d (3)                          [1, 64, 114, 114]    [1, 64, 57, 57]      --                   --
│    └─Sequential (4)                         [1, 64, 57, 57]      [1, 256, 57, 57]     --                   True
│    │    └─Bottleneck (0)                    [1, 64, 57, 57]      [1, 256, 57, 57]    

# Generate features 

In [76]:
# Single image load from the VOC dataset and in numpy format

image, target = voc_dataset[0]
image_array = np.array(image)

In [77]:
preprocess = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [60]:
gs = selective_search.get_selective_search()

selective_search.config(gs, image_array, strategy='q')

# returns coordinates as [x1, y1, x2, y2]
rects = selective_search.get_rects(gs)

In [ ]:
# parse annotation to this  [{'class': 'dog', 'bbox': [50, 50, 200, 200]}, {'class': 'cat', 'bbox': [220, 220, 350, 350]}]


In [88]:
annotations = []
for obj in target['annotation']['object']:
    annotations.append({
        'class': obj['name'],
        'bbox': [int(obj['bndbox']['xmin']), int(obj['bndbox']['ymin']), int(obj['bndbox']['xmax']), int(obj['bndbox']['ymax'])]
    })
    # print(obj['name'], obj['bndbox'])

In [89]:
annotations

[{'class': 'horse', 'bbox': [53, 87, 471, 420]},
 {'class': 'person', 'bbox': [158, 44, 289, 167]}]

In [82]:
target['annotation']

{'annotation': {'folder': 'VOC2012',
  'filename': '2008_000008.jpg',
  'source': {'database': 'The VOC2008 Database',
   'annotation': 'PASCAL VOC2008',
   'image': 'flickr'},
  'size': {'width': '500', 'height': '442', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'horse',
    'pose': 'Left',
    'truncated': '0',
    'occluded': '1',
    'bndbox': {'xmin': '53', 'ymin': '87', 'xmax': '471', 'ymax': '420'},
    'difficult': '0'},
   {'name': 'person',
    'pose': 'Unspecified',
    'truncated': '1',
    'occluded': '0',
    'bndbox': {'xmin': '158', 'ymin': '44', 'xmax': '289', 'ymax': '167'},
    'difficult': '0'}]}}

In [90]:
rects[0]

array([412, 236, 442, 286])

In [91]:
iou = calculate_iou(rects[0], [53, 87, 471, 420])

In [92]:
iou

0.01077632656580025

In [78]:
# Process each region proposal
features = []

for x, y, w, h in rects:
    # Crop and preprocess the region
    region = image.crop((x, y, x + w, y + h))
    region_preprocessed = preprocess(region).unsqueeze(0)  # Add batch dimension

    # Move to GPU if available
    if torch.cuda.is_available():
        region_preprocessed = region_preprocessed.cuda()

    # Extract features
    with torch.no_grad():
        feature = modified_resnet50(region_preprocessed)
        features.append(feature.cpu().numpy().flatten())  # Flatten the features

In [81]:
annotations = []
for obj in target['annotation']['object']:
    annotations.append({
        'class': obj['name'],
        'bbox': [int(obj['bndbox']['xmin']), int(obj['bndbox']['ymin']), int(obj['bndbox']['xmax']), int(obj['bndbox']['ymax'])]
    })



(2048,)